In [2]:
import cv2
import numpy as np
#import argparse

# check size (bounding box) is square
def isSquare(siz):
    ratio = abs(siz[0] - siz[1]) / siz[0]
    #print (siz, ratio)
    if ratio < 0.1:
        return True
    else:
        return False

# chekc circle from the arc length ratio
def isCircle(cnt):
    (x,y),radius = cv2.minEnclosingCircle(cnt)
    len = cv2.arcLength(cnt, True)
    ratio = abs(len - np.pi * 2.0 * radius) / (np.pi * 2.0 * radius)
    #print(ratio)
    if ratio < 0.1:
        return True
    else:
        return False

img1 = input("Enter an Image: ")

img = cv2.imread(img1)

# Convert to gray-scale
gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
# Binary
thr,dst = cv2.threshold(gray, 60, 255, cv2.THRESH_BINARY)

kernel = np.ones((5,5), dtype = np.uint8)
kernel1 = np.ones((3,3), dtype = np.uint8)
kernel2 = np.ones((1,1), dtype = np.uint8)
# clean up
dst = cv2.morphologyEx(dst,cv2.MORPH_CLOSE,kernel)

for i in range(1):
    dst = cv2.dilate(dst, kernel1)

# find contours with hierachy
cont, hier = cv2.findContours(dst, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
#print(np.shape(cont))
# each contoure
for i in range(len(cont)):
    c = cont[i]
    h = hier[0,i]

    if h[2] == -1 and h[3] == 0:
        # no child and parent is image outer
        img = cv2.drawContours(img, cont, i, (0,0,255),-1)
    elif h[3] == 0 and hier[0,h[2]][2] == -1:
        # with child
        if isCircle(c):
            if isCircle(cont[h[2]]):
                # double circle
                img = cv2.drawContours(img, cont, i, (0,255,0),-1)
        else:
            # 1 child and shape bounding box is not squre 
            if not isSquare(cv2.minAreaRect(c)[1]) and hier[0,h[2]][0] == -1 and hier[0,h[2]][1] == -1:
                img = cv2.drawContours(img, cont, i, (255,0, 0),-1)
            else:
                img = cv2.drawContours(img, cont, i, (0, 255, 255), -1)
    elif h[0] == -1 and not isCircle(c) and hier[0,h[2]][0] == -1:
        img = cv2.drawContours(img, cont, i, (255, 0, 127), -1)

cv2.namedWindow("Image",cv2.WINDOW_NORMAL)
cv2.imshow("Image", img)
#cv2.imshow("Semi", dst)

cv2.imwrite("all-parts-output.png", img)
cv2.waitKey()
cv2.destroyAllWindows()

Enter an Image: all-parts.png
